## Librerías

In [80]:
# qa_stress_test.ipynb

# --- 0. Configuración Inicial y Carga de Librerías ---
import pandas as pd
import numpy as np
import os
import json
import yaml
import time
from openai import OpenAI
# Importación de utils asumiendo que está en la misma carpeta 'notebooks/'
# Asegúrate de que utils.py contenga la clase TimerResult y el @contextmanager def timer(name): yield result
from utils import timer 

In [81]:
# Para asegurar que se muestren todas las columnas de Pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

## path

In [82]:
# >>>>> RUTA BASE PERSONALIZADA DEL PROYECTO (INTEGRACIÓN DE TU pk_) <<<<<
# Esta ruta es esencial para que el notebook encuentre los archivos
# generados por data_generator.py (data/, company_docs/, etc.)
pk_ = "/home/adseglocdom/PycharmProjects/Repositorios/bbog-kd-pruebas-mlops/stressbot/"
# Puedes ajustar esta variable si la ubicación de tu proyecto cambia.
# >>>>> FIN DE RUTA BASE PERSONALIZADA <<<<<

## open AI

In [83]:
# --- 1. Cargar Credenciales ---
print("Cargando credenciales de OpenAI...")
try:
    # Usar pk_ para la ruta de credentials.json
    with open(os.path.join('credentials.json')) as f:
        config_env = json.load(f)
    api_key = config_env["openai_key"]
    client = OpenAI(api_key=api_key)
    print("Credenciales cargadas. Cliente OpenAI inicializado.")
except FileNotFoundError:
    print(f"ERROR: 'credentials.json' no encontrado en la ruta: {os.path.join(pk_, 'credentials.json')}. Asegúrate de crearlo en la raíz de tu proyecto ('{pk_}').")
    api_key = None
    client = None
except KeyError:
    print("ERROR: 'openai_key' no encontrada en 'credentials.json'.")
    api_key = None
    client = None


Cargando credenciales de OpenAI...
Credenciales cargadas. Cliente OpenAI inicializado.


## YAML Prompts

In [84]:
# --- 2. Cargar Prompts del Reporte Unificado ---
print("\nCargando prompts desde prompts.yml...")
# Usar pk_ para la ruta de prompts.yml
prompts_path = os.path.join('prompts.yml')
unified_report_prompts = {}
with timer("Carga de prompts y queries") as t_prompts: # Timer para esta etapa
    try:
        with open(prompts_path, 'r', encoding='utf-8') as file:
            all_prompts = yaml.safe_load(file)
        unified_report_prompts = {k: v for k, v in all_prompts.items() if k.startswith('unified_report_')}
        print(f"Prompts de reporte unificado cargados: {list(unified_report_prompts.keys())}")
    except FileNotFoundError:
        print(f"ERROR: '{prompts_path}' no encontrado.")
    except yaml.YAMLError as e:
        print(f"ERROR al parsear YAML: {e}")



Cargando prompts desde prompts.yml...
Prompts de reporte unificado cargados: ['unified_report_1', 'unified_report_2', 'unified_report_3', 'unified_report_4', 'unified_report_5']
[TIMER] Carga de prompts y queries: 0.022s


## Data

In [85]:
# --- 3. Cargar Datas Simuladas (10M Transacciones y 300K Financieros) y Mapeo de Empresas ---
print("\nCargando datas simuladas...")
transactions_df = pd.DataFrame()
financial_df = pd.DataFrame()
company_mapping = [] # Lista de diccionarios {original_name, sanitized_folder_name}

# Decide qué versión de datos cargar (LIGHT o FULL)
USE_LIGHT_DATA = True # Cambia a False para usar los datos grandes (10M/300K)

transactions_file = 'simulated_transactions.csv'
financial_file = 'simulated_financial_metrics.csv'

with timer("Carga de dataframes e historial") as t_data_load: # Timer para esta etapa
    try:
        # Usar pk_ para la ruta de los archivos de datos
        transactions_df = pd.read_csv(os.path.join(pk_, 'data', transactions_file))
        print(f"Data de transacciones cargada ({'LIGHT' if USE_LIGHT_DATA else 'FULL'}). Registros: {len(transactions_df)}")
    except FileNotFoundError:
        print(f"ERROR: '{transactions_file}' no encontrado en la ruta: '{os.path.join(pk_, 'data')}/'. Ejecuta data_generator.py (o _light.py).")

    try:
        # Usar pk_ para la ruta de los archivos de datos
        financial_df = pd.read_csv(os.path.join(pk_, 'data', financial_file))
        print(f"Data financiera cargada ({'LIGHT' if USE_LIGHT_DATA else 'FULL'}). Registros: {len(financial_df)}")
    except FileNotFoundError:
        print(f"ERROR: '{financial_file}' no encontrado en la ruta: '{os.path.join(pk_, 'data')}/'. Ejecuta data_generator.py (o _light.py).")

    # Cargar el mapeo de empresas
    try:
        # Usar pk_ para la ruta del mapeo de empresas
        with open(os.path.join(pk_, 'data', 'company_mapping.json'), 'r', encoding='utf-8') as f:
            company_mapping = json.load(f)
        print(f"Mapeo de {len(company_mapping)} empresas cargado.")
    except FileNotFoundError:
        print(f"ERROR: 'company_mapping.json' no encontrado en la ruta: '{os.path.join(pk_, 'data')}/'. Ejecuta data_generator.py (o _light.py).")
    except json.JSONDecodeError:
        print("ERROR: Fallo al leer 'company_mapping.json'. Asegúrate de que es un JSON válido.")



Cargando datas simuladas...
Data de transacciones cargada (LIGHT). Registros: 2000
Data financiera cargada (LIGHT). Registros: 1000
Mapeo de 25 empresas cargado.
[TIMER] Carga de dataframes e historial: 0.012s


In [86]:
transactions_df.head(1)

,transaction_id,company_name,date,amount,type,description
0,0,Salgado LLC S.A.S.,2022-04-18,8958.78,CREDIT,Doloribus eligendi quia temporibus necessitati...


In [87]:
financial_df.head(1)

,financial_id,company_name,year,revenue,profit,liquidity_ratio,debt_equity_ratio,cash_flow
0,0,Salgado LLC S.A.S.,2019,65019661.26,7050981.16,1.91,1.06,3295108.9


## FLOW

In [88]:
# --- 4. Función para Cargar Contenido de Documentos Simulados por Empresa (Tu versión adaptada) ---
def load_company_documents(sanitized_folder_name):
    """Carga el contenido textual simulado de los documentos (previamente PDFs convertidos a TXT)
       para una empresa usando su nombre de carpeta sanitizado."""
    doc_contents = {}
    # Usar pk_ para la ruta base de los documentos de las empresas
    base_path = os.path.join(pk_, 'company_docs', sanitized_folder_name) # 
    if not os.path.exists(base_path):
        print(f"Advertencia: Carpeta de documentos no encontrada para '{sanitized_folder_name}' en '{base_path}'") # 
        return doc_contents

    # Este timer imprime su tiempo, pero su métrica se capturará indirectamente
    # en "Creación de PineconeManagers (carga de documentos)" de run_unified_report_flow.
    with timer(f"Carga documentos para {sanitized_folder_name}") as t_doc_load: # 
        for doc_type in ['gestion', 'sectorial', 'financiero']: # 
            file_path = os.path.join(base_path, f'{doc_type}.txt') # 
            if os.path.exists(file_path): # 
                with open(file_path, 'r', encoding='utf-8') as f: # 
                    doc_contents[doc_type] = f.read() # 
            else:
                doc_contents[doc_type] = "" # Vacío si no se encuentra el archivo 
        return doc_contents # 

# --- 5. Lógica Principal de Generación de Reporte (Emulación del main.py) ---

def run_unified_report_flow(company_original_name, company_sanitized_folder_name, report_prompt_key, user_query=""):
    """
    Ejecuta un flujo simulado de generación de reporte unificado para una empresa.
    Mide tiempos y recolecta métricas.
    """ # 
    # Asegúrate de que client, unified_report_prompts, company_mapping, transactions_df, financial_df
    # estén definidos globalmente o pasados como argumentos.
    # Si no, esta comprobación fallará.
    if 'client' not in globals() or client is None or \
       'unified_report_prompts' not in globals() or not unified_report_prompts or \
       'company_mapping' not in globals() or not company_mapping or \
       'transactions_df' not in globals() or transactions_df.empty or \
       'financial_df' not in globals() or financial_df.empty: # 
        print("ERROR: Variables globales (client, prompts, mapping, dataframes) no inicializadas. Abortando.") # 
        return {"status": "failed", "error": "Setup incomplete"} # 

    print(f"\n--- Ejecutando flujo para {company_original_name} ({report_prompt_key}) ---") # 
    metrics = {
        "status": "success", # 
        "total_execution_time": 0.0, # 
        "llm_input_tokens": 0, # 
        "llm_output_tokens": 0, # 
        "data_processed_tx_rows": 0, # 
        "data_processed_fin_rows": 0, # 
        "llm_api_latency": 0.0, # Latencia de la llamada al LLM aislada 
        "timer_metrics": {} # Para guardar los tiempos de cada [TIMER] 
    }
    # Timer principal para el flujo completo
    with timer("Tiempo total conversación") as total_timer_result: # 
        # --- [TIMER] Normalización y decisión de flujo: (SIMULADO)
        with timer("Normalización y decisión de flujo") as t_norm_result: # 
            time.sleep(0.001) # Pequeña simulación de procesamiento 
            # >>>>> AÑADE ESTAS LÍNEAS PARA DEPURACIÓN <<<<<
            # print(f"DEBUG_NORM: t_norm_result object: {t_norm_result}")
            # if t_norm_result is not None:
            #     print(f"DEBUG_NORM: t_norm_result.elapsed_time: {t_norm_result.elapsed_time}")
            # else:
            #     print("DEBUG_NORM: t_norm_result es None.")
            # >>>>> FIN DE LÍNEAS DE DEPURACIÓN <<<<<
            metrics["timer_metrics"]["Normalización y decisión de flujo"] = t_norm_result.elapsed_time # 
        # --- [TIMER] Inicialización cliente e historial: (SIMULADO)
        with timer("Inicialización cliente e historial") as t_init_result: # 
            time.sleep(0.01) # Pequeña simulación 
            # >>>>> AÑADE ESTAS LÍNEAS PARA DEPURACIÓN <<<<<
            # print(f"DEBUG_INIT: t_init_result object: {t_init_result}")
            # if t_init_result is not None:
            #     print(f"DEBUG_INIT: t_init_result.elapsed_time: {t_init_result.elapsed_time}")
            # else:
            #     print("DEBUG_INIT: t_init_result es None.")
            # >>>>> FIN DE LÍNEAS DE DEPURACIÓN <<<<<
            metrics["timer_metrics"]["Inicialización cliente e historial"] = t_init_result.elapsed_time # 

        # --- [TIMER] Configuración y modelos: (SIMULADO)
        with timer("Configuración y modelos") as t_config_result: # 
            time.sleep(00.05) # Simulación de tiempo para configuración 
            # >>>>> AÑADE ESTAS LÍNEAS PARA DEPURACIÓN <<<<<
            # print(f"DEBUG_CONFIG: t_config_result object: {t_config_result}")
            # if t_config_result is not None:
            #     print(f"DEBUG_CONFIG: t_config_result.elapsed_time: {t_config_result.elapsed_time}")
            # else:
            #     print("DEBUG_CONFIG: t_config_result es None.")
            # >>>>> FIN DE LÍNEAS DE DEPURACIÓN <<<<<
            metrics["timer_metrics"]["Configuración y modelos"] = t_config_result.elapsed_time # 
        # --- [TIMER] Creación de PineconeManagers (incluye carga de docs simulada):
        with timer("Creación de PineconeManagers (carga de documentos)") as t_pinecone_init_result: # 
            company_docs = load_company_documents(company_sanitized_folder_name) # 
            # >>>>> AÑADE ESTAS LÍNEAS PARA DEPURACIÓN <<<<<
            # print(f"DEBUG_PINECONE: t_pinecone_init_result object: {t_pinecone_init_result}")
            # if t_pinecone_init_result is not None:
            #     print(f"DEBUG_PINECONE: t_pinecone_init_result.elapsed_time: {t_pinecone_init_result.elapsed_time}")
            # else:
            #     print("DEBUG_PINECONE: t_pinecone_init_result es None.")
            # >>>>> FIN DE LÍNEAS DE DEPURACIÓN <<<<<
            metrics["timer_metrics"]["Creación de PineconeManagers (carga de documentos)"] = t_pinecone_init_result.elapsed_time # 
        # --- Preparar el Prompt Final para el LLM ---
        current_report_prompt_template = unified_report_prompts.get(report_prompt_key) # 
        if not current_report_prompt_template: # 
            print(f"ERROR: Prompt '{report_prompt_key}' no encontrado.") # 
            metrics["status"] = "failed" # 
            return metrics # 
        # --- [TIMER] Filtrado de dataframes por empresa (simulando query a BD) ---
        with timer("Filtrado de dataframes por empresa (simulando query a BD)") as t_filter_df_result: # 
            company_transactions_df = transactions_df[transactions_df['company_name'] == company_original_name].copy() # 
            company_financial_df = financial_df[financial_df['company_name'] == company_original_name].copy() # 
            metrics["data_processed_tx_rows"] = len(company_transactions_df) # 
            metrics["data_processed_fin_rows"] = len(company_financial_df) # 
            # >>>>> AÑADE ESTAS LÍNEAS PARA DEPURACIÓN <<<<<
            # print(f"DEBUG_FILTER: t_filter_df_result object: {t_filter_df_result}")
            # if t_filter_df_result is not None:
            #     print(f"DEBUG_FILTER: t_filter_df_result.elapsed_time: {t_filter_df_result.elapsed_time}")
            # else:
            #     print("DEBUG_FILTER: t_filter_df_result es None.")
            # >>>>> FIN DE LÍNEAS DE DEPURACIÓN <<<<<
            metrics["timer_metrics"]["Filtrado de dataframes por empresa (simulando query a BD)"] = t_filter_df_result.elapsed_time # 

        # --- [TIMER] Búsqueda similitud reporte unificado (simulando Pinecone/RAG):
        with timer("Búsqueda similitud reporte unificado") as t_rag_search_result: # 
            context_from_docs = company_docs.get('gestion', '') + "\n\n" + \
                                company_docs.get('sectorial', '') + "\n\n" + \
                                company_docs.get('financiero', '') # 
            time.sleep(0.01 + len(context_from_docs) / 1000000.0) # 
            # Esta ya la teníamos, la dejo para consistencia
            # print(f"DEBUG_RAG: Tipo de t_rag_search_result después del bloque timer: {type(t_rag_search_result)}")
            # if t_rag_search_result is not None:
            #     print(f"DEBUG_RAG: Valor de t_rag_search_result.elapsed_time: {t_rag_search_result.elapsed_time}")
            # else:
            #     print("DEBUG_RAG: t_rag_search_result es None.")
            metrics["timer_metrics"]["Búsqueda similitud reporte unificado"] = t_rag_search_result.elapsed_time # 

        # --- Rellenar los placeholders del prompt ---
        simulated_products_list = "Préstamos Comerciales, Créditos de Liquidez, Cuentas de Ahorro, CDT." # 
        simulated_sector = "Tecnología y Servicios Financieros" # 

        df_desem_pag_cast_md_str = company_transactions_df.head(5).to_markdown(index=False) # 
        df_perfilador_md_str = company_financial_df.head(5).to_markdown(index=False) # 

        response_va_simulated = "Según Valora Analitik, la empresa ha invertido en IA para optimizar procesos bancarios." # 
        response_pp_simulated = "En Primera Página se destacó la expansión regional de la empresa en el último año." # 

        try:
            formatted_prompt = current_report_prompt_template.format(
                company_name=company_original_name, # 
                user_request=user_query if user_query else f"Genera un reporte unificado para {company_original_name} basado en los datos proporcionados.", # 
                management_report=company_docs.get('gestion', 'No disponible'), # 
                sector=simulated_sector, # 
                sector_report=company_docs.get('sectorial', 'No disponible'), # 
                financial_report=company_docs.get('financiero', 'No disponible'), # 
                response_va=response_va_simulated, # 
                response_pp=response_pp_simulated, # 
                df_desem_pag_cast_md=df_desem_pag_cast_md_str, # 
                df_perfilador_md=df_perfilador_md_str, # 
                products_list=simulated_products_list # 
            )
        except KeyError as e:
            print(f"ERROR: Placeholder '{e}' no encontrado en el prompt '{report_prompt_key}'. Revisa tu prompts.yml.") # 
            metrics["status"] = "failed" # 
            return metrics # 

        full_messages = [
            {"role": "system", "content": formatted_prompt}, # 
            {"role": "user", "content": user_query if user_query else f"Genera el reporte unificado para {company_original_name}."} # 
        ]

        # --- [TIMER] Generación prompt + invocación LLM (reporte) ---
        llm_invocation_start_time = time.perf_counter() # 
        try:
            completion = client.chat.completions.create(
                model="gpt-4", # Asegúrate de tener acceso a este modelo 
                temperature=0.0, # 
                messages=full_messages # 
            )
            metrics["llm_api_latency"] = time.perf_counter() - llm_invocation_start_time # 
            response_content = completion.choices[0].message.content # 
            metrics["llm_input_tokens"] = completion.usage.prompt_tokens # 
            metrics["llm_output_tokens"] = completion.usage.completion_tokens # 
            # Usar la misma métrica de latencia de API para el timer_metrics
            metrics["timer_metrics"]["Generación prompt + invocación LLM (reporte)"] = metrics["llm_api_latency"] # 
            print(f"Respuesta del LLM generada (primeros 200 chars): {response_content[:200]}...") # 
        except Exception as e:
            print(f"ERROR en invocación LLM: {e}") # 
            metrics["status"] = "failed" # 
            metrics["error"] = str(e) # 
            # Registrar el tiempo del intento incluso si falla
            metrics["timer_metrics"]["Generación prompt + invocación LLM (reporte)"] = time.perf_counter() - llm_invocation_start_time # 
            response_content = "ERROR" # 
    # Captura el tiempo total del contexto principal al salir del 'with timer'
    metrics["total_execution_time"] = total_timer_result.elapsed_time # 

    print(f"\n--- Métricas Finales para {company_original_name} ({report_prompt_key}) ---") # 
    print(f"Tiempo Total de Ejecución: {metrics['total_execution_time']:.3f}s") # 
    print(f"Latencia de Invocación LLM (aislada): {metrics['llm_api_latency']:.3f}s") # 
    print(f"Tokens de Entrada LLM: {metrics['llm_input_tokens']}") # 
    print(f"Tokens de Salida LLM: {metrics['llm_output_tokens']}") # 
    print(f"Volumen de Transacciones procesadas: {metrics['data_processed_tx_rows']} filas") # 
    print(f"Volumen de Financieros procesados: {metrics['data_processed_fin_rows']} filas") # 
    print(f"Estado del flujo: {metrics['status']}") # 
    print("Tiempos por subproceso:") # 
    for k, v in metrics["timer_metrics"].items(): # 
        if v is not None: # Asegurar que el valor no sea None antes de formatear 
            print(f" - {k}: {v:.3f}s") # 
        else:
            print(f" - {k}: N/A (tiempo no capturado)") # 

    return {"status": metrics["status"], "response_content": response_content, "metrics": metrics}


## Latencia

In [89]:
company_mapping

[{'original_name': 'Ibarra Ltd S.A.S.',
  'sanitized_folder_name': 'empresa_1_Ibarra_Ltd_SAS'},
 {'original_name': 'Castro Group S.A.S.',
  'sanitized_folder_name': 'empresa_2_Castro_Group_SAS'},
 {'original_name': 'Suárez-Gallego S.A.S.',
  'sanitized_folder_name': 'empresa_3_Suarez-Gallego_SAS'},
 {'original_name': 'Rojas, Ramírez and Castro S.A.S.',
  'sanitized_folder_name': 'empresa_4_Rojas_Ramirez_and_Castro_SAS'},
 {'original_name': 'Moreno Group S.A.S.',
  'sanitized_folder_name': 'empresa_5_Moreno_Group_SAS'},
 {'original_name': 'Romero PLC S.A.S.',
  'sanitized_folder_name': 'empresa_6_Romero_PLC_SAS'},
 {'original_name': 'Henao, Ríos and Martínez S.A.S.',
  'sanitized_folder_name': 'empresa_7_Henao_Rios_and_Martinez_SAS'},
 {'original_name': 'Ortega PLC S.A.S.',
  'sanitized_folder_name': 'empresa_8_Ortega_PLC_SAS'},
 {'original_name': 'Quintero and Sons S.A.S.',
  'sanitized_folder_name': 'empresa_9_Quintero_and_Sons_SAS'},
 {'original_name': 'Palacio Ltd S.A.S.',
  'saniti

In [90]:
unified_report_prompts

{'unified_report_1': 'Eres un analista experto en la elaboración de informes gerenciales para el área comercial del Banco de Bogotá. Tu objetivo, juanto con otros analistas, es generar un informe estratégico comercial (detallado) que busca profundizar relaciones con el cliente {company_name}. Tienes asignado uno de los puntos del informe general.\nPara lograr tu tarea dentro de la división (entre analistas) de objetivo, sigue esta estructura, asegurando calidad analítica y redacción clara. Importante evitar repetir datos entre secciones (especialmente cifras, conclusiones y diagnósticos):\n\n1. Descripción general de la empresa: a partir de toda la información suministrada realiza un resumen que incluya la actividad de la empresa {company_name}; sus principales productos y servicios (realiza una lista que contenga descripciones breves para cada elemento); quiénes son los clientes de la empresa {company_name}; información de los competidores directos de la empresa {company_name} en el m

In [91]:
# --- 6. Ejecutar Pruebas (Ejemplo) ---
# Primero, asegúrate de haber ejecutado data_generator.py (o _light.py)
# para que los archivos CSV, TXT y company_mapping.json existan. 
if not company_mapping: # 
    print("No se pudo cargar el mapeo de empresas. Asegúrate de ejecutar el generador de datos primero.") # 
else:
    test_company_data = company_mapping[0] # Tomamos la primera empresa del mapeo para la prueba de ejemplo 
    test_company_original_name = test_company_data["original_name"] # 
    test_company_sanitized_folder_name = test_company_data["sanitized_folder_name"] # 

    print(f"\nRealizando una prueba de ejemplo para la empresa: {test_company_original_name}") # 
    if 'unified_report_1' in unified_report_prompts: # 
        results = run_unified_report_flow(
            company_original_name=test_company_original_name, # 
            company_sanitized_folder_name=test_company_sanitized_folder_name, # 
            report_prompt_key='unified_report_1', # 
            user_query="Genera el resumen general de la empresa con los datos proporcionados." # 
        )
        print("\n--- Resultados Detallados de la Prueba de Ejemplo ---") # 
        print(json.dumps(results, indent=2)) # 
    else:
        print("El prompt 'unified_report_1' no está disponible en prompts.yml. Por favor, revisa tus prompts.") # 



Realizando una prueba de ejemplo para la empresa: Ibarra Ltd S.A.S.

--- Ejecutando flujo para Ibarra Ltd S.A.S. (unified_report_1) ---
[TIMER] Normalización y decisión de flujo: 0.001s
[TIMER] Inicialización cliente e historial: 0.010s
[TIMER] Configuración y modelos: 0.050s
[TIMER] Carga documentos para empresa_1_Ibarra_Ltd_SAS: 0.000s
[TIMER] Creación de PineconeManagers (carga de documentos): 0.000s
[TIMER] Filtrado de dataframes por empresa (simulando query a BD): 0.001s
[TIMER] Búsqueda similitud reporte unificado: 0.011s
Respuesta del LLM generada (primeros 200 chars): 1. Descripción general de la empresa:

Ibarra Ltd S.A.S. es una empresa que ha consolidado su presencia en el mercado asperiores, experimentando un crecimiento del 7% en sus operaciones en el último a...
[TIMER] Tiempo total conversación: 20.646s

--- Métricas Finales para Ibarra Ltd S.A.S. (unified_report_1) ---
Tiempo Total de Ejecución: 20.646s
Latencia de Invocación LLM (aislada): 20.567s
Tokens de Entrada LL

## Pruebas de Stress

In [92]:
company_mapping[0]

{'original_name': 'Ibarra Ltd S.A.S.',
 'sanitized_folder_name': 'empresa_1_Ibarra_Ltd_SAS'}

In [93]:
all_test_results=[] # 
num_companies_to_test=3 # 
for i in range(num_companies_to_test): # 
    company_data=company_mapping[i] # 
    company_original_name=company_data["original_name"] # 
    company_sanitized_folder_name=company_data["sanitized_folder_name"] # 
    for prompt_key in ["unified_report_1"]: # 
        if prompt_key in unified_report_prompts: # 
            result=run_unified_report_flow(
                company_original_name=company_original_name, # 
                company_sanitized_folder_name=company_sanitized_folder_name, # 
                report_prompt_key=prompt_key, # 
                user_query=f"Genera el reporte de {prompt_key} para {company_original_name}" # 
            )
            all_test_results.append({"Company Name":company_original_name, "Prompt": prompt_key, "Metrics": result}) # 
        else:
            print(f"Advertencia: El prompt {prompt_key} no fue encontrado") # 


--- Ejecutando flujo para Ibarra Ltd S.A.S. (unified_report_1) ---
[TIMER] Normalización y decisión de flujo: 0.001s
[TIMER] Inicialización cliente e historial: 0.010s
[TIMER] Configuración y modelos: 0.050s
[TIMER] Carga documentos para empresa_1_Ibarra_Ltd_SAS: 0.000s
[TIMER] Creación de PineconeManagers (carga de documentos): 0.000s
[TIMER] Filtrado de dataframes por empresa (simulando query a BD): 0.001s
[TIMER] Búsqueda similitud reporte unificado: 0.011s
Respuesta del LLM generada (primeros 200 chars): # Unified Report 1: Ibarra Ltd S.A.S.

## 1. Descripción general de la empresa

Ibarra Ltd S.A.S. es una empresa que ha consolidado su presencia en el mercado asperiores, experimentando un crecimiento...
[TIMER] Tiempo total conversación: 34.404s

--- Métricas Finales para Ibarra Ltd S.A.S. (unified_report_1) ---
Tiempo Total de Ejecución: 34.404s
Latencia de Invocación LLM (aislada): 34.327s
Tokens de Entrada LLM: 1886
Tokens de Salida LLM: 571
Volumen de Transacciones procesadas

In [94]:
all_test_results[0]

{'Company Name': 'Ibarra Ltd S.A.S.',
 'Prompt': 'unified_report_1',
 'Metrics': {'status': 'success',
  'response_content': '# Unified Report 1: Ibarra Ltd S.A.S.\n\n## 1. Descripción general de la empresa\n\nIbarra Ltd S.A.S. es una empresa que ha consolidado su presencia en el mercado asperiores, experimentando un crecimiento del 7% en sus operaciones en el último año. La compañía se enfoca en la prestación de servicios y productos en el sector de Tecnología y Servicios Financieros, con proyectos estratégicos centrados en Iste ducimus nobis sequi distinctio perspiciatis animi nostrum quisquam.\n\nLa empresa cuenta con una fuerza laboral de 359 empleados y ha lanzado una iniciativa de sostenibilidad que busca reducir la huella de carbono en un 15%. Sus principales competidores en el mercado son Vera Inc y Cerón Group. \n\nEn cuanto a su cadena de suministros, Ibarra Ltd S.A.S. ha invertido en Inteligencia Artificial para optimizar procesos bancarios, según información de Valora Anali

In [95]:
all_test_results[1]

{'Company Name': 'Castro Group S.A.S.',
 'Prompt': 'unified_report_1',
 'Metrics': {'status': 'success',
  'response_content': '# Unified Report 1: Castro Group S.A.S.\n\n## 1. Descripción general de la empresa\n\nCastro Group S.A.S. es una empresa que ha experimentado un crecimiento del 15% en sus operaciones en el último año, consolidando su presencia en el mercado. La compañía se ha enfocado en proyectos estratégicos que incluyen la optimización de procesos bancarios a través de la inversión en Inteligencia Artificial (IA) y la expansión regional. Actualmente, la fuerza laboral de la empresa asciende a 174 empleados.\n\nLa competencia principal de Castro Group S.A.S. incluye a Montes-Rivera y Mena, Sánchez and Murillo. La empresa ha lanzado una iniciativa de sostenibilidad que busca reducir la huella de carbono en un 29%. Además, el directorio ha aprobado un plan de inversión de $ 3,017,991 USD para nuevas tecnologías.\n\nEn términos de su cadena de suministros, Castro Group S.A.S. 

In [96]:
all_test_results[2]

{'Company Name': 'Suárez-Gallego S.A.S.',
 'Prompt': 'unified_report_1',
 'Metrics': {'status': 'success',
  'response_content': '# Informe Estratégico Comercial: Suárez-Gallego S.A.S.\n\n## 1. Descripción General de la Empresa\n\nSuárez-Gallego S.A.S. es una empresa consolidada en el sector de Tecnología y Servicios Financieros. En el último año, ha experimentado un crecimiento del 19% en sus operaciones, lo que ha fortalecido su presencia en el mercado. La compañía ha centrado sus proyectos estratégicos en Accusantium veniam consequuntur quidem ratione mollitia delectus eum rem possimus autem sapiente quasi, lo que ha contribuido a su crecimiento. \n\nLa empresa cuenta con una fuerza laboral de 670 empleados y ha lanzado una iniciativa de sostenibilidad que busca reducir la huella de carbono en un 14%. Las inversiones futuras de Suárez-Gallego S.A.S. se centran en Delectus modi sint quas odit, y el directorio ha aprobado un plan de inversión de $ 335,858 USD para nuevas tecnologías.\

## Stress Paralelizado 

In [97]:
from concurrent.futures import ThreadPoolExecutor

all_test_results_parallel=[] # 
num_companies_to_test=3 # 
with ThreadPoolExecutor(max_workers=num_companies_to_test) as executor: # 
    futures=[] # 
    for i in range(num_companies_to_test): # 
        company_data=company_mapping[i] # 
        company_original_name=company_data["original_name"] # 
        company_sanitized_folder_name=company_data["sanitized_folder_name"] # 
        for prompt_key in ["unified_report_1"]: # 
            if prompt_key in unified_report_prompts: # 
                future= executor.submit(run_unified_report_flow, 
                company_original_name=company_original_name, # 
                company_sanitized_folder_name=company_sanitized_folder_name, # 
                report_prompt_key=prompt_key, # 
                user_query=f"Genera el reporte de {prompt_key} para {company_original_name}" # 
                )
                futures.append((future, company_original_name, prompt_key)) # 
            else:
                print(f"Advertencia: El prompt {prompt_key} no fue encontrado") # 
    for future, company_name, prompt_key in futures: # 
        try:
            result= future.result() # 
            all_test_results_parallel.append({
                "Company_name": company_name, # 
                "Prompt_key": prompt_key, # 
                "Metrics": result # 
            })
        except Exception as exec:
            all_test_results_parallel.append({
                "Company_name": company_name, # 
                "Prompt_key": prompt_key, # 
                "Metrics": {"status":"failed","error":str(exec),"time":0.0} # 
            })



--- Ejecutando flujo para Ibarra Ltd S.A.S. (unified_report_1) ---

--- Ejecutando flujo para Castro Group S.A.S. (unified_report_1) ---

--- Ejecutando flujo para Suárez-Gallego S.A.S. (unified_report_1) ---
[TIMER] Normalización y decisión de flujo: 0.001s
[TIMER] Normalización y decisión de flujo: 0.001s
[TIMER] Normalización y decisión de flujo: 0.001s
[TIMER] Inicialización cliente e historial: 0.010s
[TIMER] Inicialización cliente e historial: 0.010s
[TIMER] Inicialización cliente e historial: 0.010s
[TIMER] Configuración y modelos: 0.050s
[TIMER] Carga documentos para empresa_1_Ibarra_Ltd_SAS: 0.000s
[TIMER] Creación de PineconeManagers (carga de documentos): 0.000s
[TIMER] Filtrado de dataframes por empresa (simulando query a BD): 0.002s
[TIMER] Configuración y modelos: 0.051s
[TIMER] Configuración y modelos: 0.051s
[TIMER] Carga documentos para empresa_2_Castro_Group_SAS: 0.000s
[TIMER] Creación de PineconeManagers (carga de documentos): 0.000s
[TIMER] Filtrado de dataframes 

In [98]:
all_test_results_parallel[0] # 

{'Company_name': 'Ibarra Ltd S.A.S.',
 'Prompt_key': 'unified_report_1',
 'Metrics': {'status': 'success',
  'response_content': '# Unified Report 1: Ibarra Ltd S.A.S.\n\n## 1. Descripción general de la empresa\n\nIbarra Ltd S.A.S. es una empresa que ha consolidado su presencia en el mercado asperiores, experimentando un crecimiento del 7% en sus operaciones en el último año. La compañía se ha enfocado en proyectos estratégicos en Iste ducimus nobis sequi distinctio perspiciatis animi nostrum quisquam. Actualmente, la fuerza laboral de la empresa asciende a 359 empleados.\n\nLa empresa ha lanzado una iniciativa de sostenibilidad que busca reducir la huella de carbono en un 15%. Además, las inversiones futuras de Ibarra Ltd S.A.S. se centran en Dolore facere nihil nam laboriosam quaerat totam ipsa exercitationem, con un plan de inversión de $ 2,288,878 USD para nuevas tecnologías aprobado por el directorio.\n\nLos competidores directos de Ibarra Ltd S.A.S. en el mercado incluyen a Vera 

In [99]:
all_test_results_parallel[1] # 

{'Company_name': 'Castro Group S.A.S.',
 'Prompt_key': 'unified_report_1',
 'Metrics': {'status': 'success',
  'response_content': '# Descripción General de la Empresa: Castro Group S.A.S.\n\n## Actividad de la Empresa\nCastro Group S.A.S. es una empresa que ha experimentado un crecimiento del 15% en sus operaciones en el último año, consolidando su presencia en el mercado. Sus proyectos estratégicos se han enfocado en la optimización de procesos bancarios a través de la inversión en Inteligencia Artificial (IA).\n\n## Productos y Servicios\nLos principales productos y servicios de Castro Group S.A.S. incluyen soluciones tecnológicas y servicios financieros. La empresa ha invertido en nuevas tecnologías y ha lanzado una iniciativa de sostenibilidad que busca reducir la huella de carbono en un 29%.\n\n## Clientes\nLos clientes de Castro Group S.A.S. son principalmente empresas y organizaciones que requieren soluciones tecnológicas y servicios financieros.\n\n## Competidores Directos\nLo

In [100]:
all_test_results_parallel[2] # 

{'Company_name': 'Suárez-Gallego S.A.S.',
 'Prompt_key': 'unified_report_1',
 'Metrics': {'status': 'success',
  'response_content': '# Descripción General de la Empresa: Suárez-Gallego S.A.S.\n\nSuárez-Gallego S.A.S. es una empresa que ha experimentado un crecimiento del 19% en sus operaciones en el último año, consolidando su presencia en el mercado. La compañía se ha enfocado en proyectos estratégicos que buscan optimizar sus operaciones y mejorar su rendimiento. Actualmente, la fuerza laboral de la empresa asciende a 670 empleados.\n\n## Principales Productos y Servicios\n\nLa empresa se ha centrado en la implementación de nuevas tecnologías, con una inversión aprobada de $335,858 USD. Además, ha invertido en Inteligencia Artificial para optimizar procesos bancarios.\n\n## Clientes\n\nSuárez-Gallego S.A.S. ha expandido su presencia regional, lo que sugiere una base de clientes diversa y en crecimiento.\n\n## Competidores Directos\n\nLos competidores principales de Suárez-Gallego S.

## Generación y Almacenamiento de Reportes en Archivos

In [101]:
# --- NUEVA SECCIÓN: Generación de Reporte Consolidado por Empresa de Forma Paralelizada ---
print("\n--- Iniciando generación paralela de reportes consolidados por empresa ---")

# ==============================================================================
# FUNCIÓN TRABAJADORA PARA GENERAR Y GUARDAR REPORTE CONSOLIDADO POR EMPRESA (TXT)
# ==============================================================================
def generar_y_guardar_reporte_completo_paralelo(company_data, all_prompts, base_pk_path):
    """
    Función trabajadora que genera un reporte consolidado para una sola empresa
    iterando sobre todos los prompts y guardándolo como un archivo .txt.
    Diseñada para ser ejecutada en un hilo separado.
    """
    company_original_name = company_data["original_name"]
    company_sanitized_folder_name = company_data["sanitized_folder_name"]
    
    print(f"[Thread] Iniciando procesamiento consolidado para: {company_original_name}")

    consolidated_report_content = f"Reporte Consolidado para: {company_original_name}\n"
    consolidated_report_content += "="*80 + "\n"

    available_prompts = sorted([key for key in all_prompts.keys()])
    for prompt_key in available_prompts:
        print(f"[Thread] -> Ejecutando prompt '{prompt_key}' para {company_original_name}")
        
        # Llamamos a run_unified_report_flow para cada prompt
        result = run_unified_report_flow(
            company_original_name=company_original_name,
            company_sanitized_folder_name=company_sanitized_folder_name,
            report_prompt_key=prompt_key,
            user_query=f"Genera el reporte detallado de '{prompt_key}' para la empresa {company_original_name}"
        )
        
        consolidated_report_content += f"\n--- INICIO REPORTE DEL PROMPT: {prompt_key} ---\n\n"
        if result.get("status") == "success" and "response_content" in result:
            consolidated_report_content += result["response_content"] + "\n"
        else:
            error_message = result.get("error", "Contenido no generado.")
            consolidated_report_content += f"FALLO AL GENERAR REPORTE: {error_message}\n"
        consolidated_report_content += f"\n--- FIN REPORTE DEL PROMPT: {prompt_key} ---\n"
        consolidated_report_content += "-"*80 + "\n"

    # Crear el directorio de salida si no existe
    output_reports_dir = os.path.join(base_pk_path, 'data', 'generated_reports_parallel_consolidated')
    os.makedirs(output_reports_dir, exist_ok=True)
    
    # Guardar el archivo en TXT
    try:
        output_filename = f"{company_sanitized_folder_name}_consolidated_report.txt"
        output_filepath = os.path.join(output_reports_dir, output_filename)
        with open(output_filepath, 'w', encoding='utf-8') as f:
            f.write(consolidated_report_content)
        
        return f"✅ Reporte consolidado para '{company_original_name}' guardado exitosamente en {output_filename}"
    except Exception as e:
        return f"❌ ERROR al guardar el reporte consolidado para '{company_original_name}': {e}"

# ==============================================================================
# EJECUCIÓN PARALELIZADA DE LA GENERACIÓN DE REPORTES CONSOLIDADOS
# ==============================================================================
parallel_consolidated_results = []
num_companies_to_process_parallel = 3 # Define cuántas empresas procesar en paralelo

if not company_mapping:
    print("No se pudo cargar el mapeo de empresas. Asegúrate de ejecutar el generador de datos primero.")
else:
    with ThreadPoolExecutor(max_workers=num_companies_to_process_parallel) as executor:
        futures = []
        for i in range(min(num_companies_to_process_parallel, len(company_mapping))):
            company_data = company_mapping[i]
            
            future = executor.submit(
                generar_y_guardar_reporte_completo_paralelo,
                company_data,
                unified_report_prompts,
                pk_
            )
            futures.append(future)

        print("\n--- Esperando a que todos los hilos de generación consolidada terminen... ---")
        for future in futures:
            try:
                result_message = future.result()
                print(result_message)
                parallel_consolidated_results.append(result_message)
            except Exception as exec:
                print(f"❌ Un hilo de generación consolidada falló con una excepción: {exec}")
                parallel_consolidated_results.append(f"FALLO consolidado: {exec}")

print("\n--- ✅ Proceso de generación paralela de reportes consolidados completado. ---")


--- Iniciando generación paralela de reportes consolidados por empresa ---
[Thread] Iniciando procesamiento consolidado para: Ibarra Ltd S.A.S.
[Thread] -> Ejecutando prompt 'unified_report_1' para Ibarra Ltd S.A.S.

--- Ejecutando flujo para Ibarra Ltd S.A.S. (unified_report_1) ---
[Thread] Iniciando procesamiento consolidado para: Castro Group S.A.S.
[Thread] -> Ejecutando prompt 'unified_report_1' para Castro Group S.A.S.

--- Ejecutando flujo para Castro Group S.A.S. (unified_report_1) ---
[Thread] Iniciando procesamiento consolidado para: Suárez-Gallego S.A.S.
[Thread] -> Ejecutando prompt 'unified_report_1' para Suárez-Gallego S.A.S.

--- Ejecutando flujo para Suárez-Gallego S.A.S. (unified_report_1) ---

--- Esperando a que todos los hilos de generación consolidada terminen... ---
[TIMER] Normalización y decisión de flujo: 0.001s
[TIMER] Normalización y decisión de flujo: 0.001s
[TIMER] Normalización y decisión de flujo: 0.001s
[TIMER] Inicialización cliente e historial: 0.010s